In [4]:
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.models as models
import numpy as np
from PIL import Image

# -------------------------------
# 1. Load Pretrained Model (BN-Inception substitute = InceptionV3)
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

weights = models.Inception_V3_Weights.DEFAULT
model = models.inception_v3(weights=weights, aux_logits=True)
model.fc = nn.Identity()  # remove final classification layer -> 2048-d features
model = model.to(device).eval()



Using device: cuda


In [5]:

# -------------------------------
# 2. Transformations
# -------------------------------
transform = T.Compose([
    T.Resize((256, 456)),  # (H=256, W=456)
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# -------------------------------
# 3. Extract RGB Features for a Video


In [8]:
# -------------------------------
def extract_rgb_features(video_path, sample_rate=5, save_path=None):
    cap = cv2.VideoCapture(video_path)
    features_list = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # sample every Nth frame
        if frame_count % sample_rate == 0:
            # Convert from OpenCV BGR → RGB → PIL
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb)

            # Apply transforms
            img = transform(img_pil).unsqueeze(0).to(device)  # (1,3,H,W)

            with torch.no_grad():
                feat = model(img)  # shape (1, 2048)
            feat = feat.squeeze().cpu().numpy()

            features_list.append(feat)

        frame_count += 1

    cap.release()

    features = np.array(features_list)  # (num_samples, 2048)
    if save_path:
        np.save(save_path, features)
        print(f"Saved {features.shape} features to {save_path}")

    return features

# -------------------------------
# 4. Example Usage
# -------------------------------
features = extract_rgb_features(
    r"C:\Users\Alis\Desktop\Video\P01_05.MP4",
    sample_rate=5,
    save_path="P01_05_rgb.npy"
)

print("RGB Features shape:", features.shape)

Saved (15249, 2048) features to P01_05_rgb.npy
RGB Features shape: (15249, 2048)


In [4]:
import torch
from torch.hub import load_state_dict_from_url
import requests
from io import BytesIO


In [5]:
def load_weights_with_requests(url, map_location="cpu"):
    r = requests.get(url)
    r.raise_for_status()
    state_dict = torch.load(BytesIO(r.content), map_location=map_location)
    return state_dict